"Look, sir.  Look, sir.  Mr. Knox, sir.  
Let's do tricks with bricks and blocks, sir.  
Let's do tricks with chicks and clocks, sir.  

First, I'll make a quick trick brick stack.  
Then I'll make a quick trick block stack."  

- Fox in Socks, by Dr. Seuss


What follows is two sets of functions you can combine to first get some data from an API and then do something with it (usually using another API).

Your assignment is to pick some functions, combine them (as shown in the example below), to do things like:  
   * text Chuck Norris jokes to Steve
   
   

# Data-getting bricks

In [21]:
def get_another_chuck_norris_joke():
    """Returns a Chuck Norris joke as a 100% ASCII string (as Chuck Norris intended)."""
    import requests
    r = requests.get("http://api.icndb.com/jokes/random?exclude=[explicit]")
    escaped_joke = r.json()['value']['joke'] # The escaped version may contain &quot; instead of an ASCII quote character.
    import html
    ascii_joke = html.unescape(escaped_joke)
    return ascii_joke

In [58]:
def max_h2s_so_far_today(station='Liberty'):
    """Get the highest concentration of hydrogen sulfide measured at a particular station (defaulting to Liberty) in Allegheny County so far today.  See the air-quality-exploration notebook for a detailed description of how this function was put together."""
    def query_resource(site,query):
        """Use the datastore_search_sql API endpoint to query a CKAN resource."""
        import ckanapi
        ckan = ckanapi.RemoteCKAN(site)
        response = ckan.action.datastore_search_sql(sql=query)
        return response['records']

    site = "https://data.wprdc.org"
    daily_air_quality_resource_id = "967f1285-f8fb-4785-9673-64a8ae47588d"
    subset = query_resource(site,
    query="SELECT site, datetime, h2s FROM \"{}\" WHERE site = '{}' AND stat = 'Max' ORDER BY datetime DESC LIMIT 1".format(daily_air_quality_resource_id,station))
    reading = subset[0]['h2s']
    if reading is None:
        return "No reading. Sorry!"
    else:
        return "{} ppm".format(reading)

In [ ]:
def get_311_requests(category=None, neighborhood=None, time_range='most_recent_one'):
    """Get 311 complaints for the City of Pittsburgh based on category, neighborhood, and time range.
    
    Returns a string describing the request or requests (aggregating for multiple requests)."""
    def query_resource(site,query):
        """Use the datastore_search_sql API endpoint to query a CKAN resource."""
        import ckanapi
        ckan = ckanapi.RemoteCKAN(site)
        response = ckan.action.datastore_search_sql(sql=query)
        return response['records']
        site = "https://data.wprdc.org"
    three_one_one_resource_id = "76fda9d0-69be-4dd5-8108-0de7907fc5a4"
    subset = query_resource(site,
    query="SELECT * FROM \"{}\" WHERE \"REQUEST_TYPE\" = '{}' ORDER BY \"CREATED_ON\" DESC LIMIT 1".format(three_one_one_resource_id,category))


# Data-putting bricks

### Zero-configuration data-putting bricks

In [74]:
def print_in_random_ascii_art_font(msg):
    from art import text2art, tprint
    tprint(msg,"random")
def print_long_message_in_ascii_art_font(long_msg,font=None):
    from art import text2art, tprint
    for word in long_msg.split(' '):
        if font is None:
            tprint(word)
        else:
            tprint(word,font)

### Data-putting bricks that require set-up

#### Twilio

# Make a brick stack

In [32]:
print(get_another_chuck_norris_joke())


The Bermuda Triangle used to be the Bermuda Square, until Chuck Norris Roundhouse kicked one of the corners off.


In [60]:
print_in_random_ascii_art_font(max_h2s_so_far_today())

 ____    ____  ____ _____    ____  ____  _     
/  _ \  /  _ \/  _ \\__  \  /  __\/  __\/ \__/|
| / \|  | / \|| / \|  /  |  |  \/||  \/|| |\/||
| \_/|__| \_/|| \_/| _\  |  |  __/|  __/| |  ||
\____/\/\____/\____//____/  \_/   \_/   \_/  \|
                                               



In [75]:
print_long_message_in_ascii_art_font(get_another_chuck_norris_joke())

__        __ _                  
\ \      / /| |__    ___  _ __  
 \ \ /\ / / | '_ \  / _ \| '_ \ 
  \ V  V /  | | | ||  __/| | | |
   \_/\_/   |_| |_| \___||_| |_|
                                

  ____  _                   _    
 / ___|| |__   _   _   ___ | | __
| |    | '_ \ | | | | / __|| |/ /
| |___ | | | || |_| || (__ |   < 
 \____||_| |_| \__,_| \___||_|\_\
                                 

 _   _                     _      
| \ | |  ___   _ __  _ __ (_) ___ 
|  \| | / _ \ | '__|| '__|| |/ __|
| |\  || (_) || |   | |   | |\__ \
|_| \_| \___/ |_|   |_|   |_||___/
                                  

                       _      
 ___   ___  _ __    __| | ___ 
/ __| / _ \| '_ \  / _` |/ __|
\__ \|  __/| | | || (_| |\__ \
|___/ \___||_| |_| \__,_||___/
                              

 _        
(_) _ __  
| || '_ \ 
| || | | |
|_||_| |_|
          

 _      _      
| |__  (_) ___ 
| '_ \ | |/ __|
| | | || |\__ \
|_| |_||_||___/
               

 _                              
|

# The end

There's lots of other sample Jupyter notebooks to explore at [https://github.com/WPRDC/api-training](https://github.com/WPRDC/api-training) and [https://github.com/WPRDC/list-of-WPRDC-relevant-tools-and-analyses](https://github.com/WPRDC/list-of-WPRDC-relevant-tools-and-analyses).

When you're done with this notebook, uncomment the line in the block below (by deleting the leading "#" character), and run the code.

In [79]:
import the_end
#the_end.conclusion()